# 2024 Marathon Training Data Transformation and Pre-Processing for PBI

## Import Packages

In [ ]:
# Import Packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Show All Columns and Rows when viewing Dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Running Workout Data
This dataset solely consists of my running data. Moreover, each row represents 1 run and all of the metrics associated with that run.

In [ ]:
# Import Dataset
dataframe = pd.read_csv("Running_Data_20241111.csv")
dataframe.head()

In [ ]:
# Create a copy of the original dataset
df = dataframe.copy()
df.head()

In [ ]:
# Dropping columns that consist entirely of null values
print(df.shape)
print(df.dropna(axis = 1, how='all').shape) # Compare shape after dropping null columns

df_cleaned = df.dropna(axis = 1, how='all')

# View Results
df_cleaned.head()

In [ ]:
# Add '_' into Column Headers
df_cleaned.columns = df_cleaned.columns.str.replace(' ', '_')

# Remove special characters from column names
df_cleaned.columns = df_cleaned.columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)

# View Results
df_cleaned.head()

### Feature Engineering



In [ ]:
# Create 'Distance_Group' Column to group runs into mileage buckets
bins = [0, 3, 5, 7, 10, 13, float('inf')]
labels = ['0-3 miles', '3-5 miles', '5-7 miles', '7-10 miles', '10-13 miles', '13+ miles']

# Use Distance Value to assign Distance Group
df_cleaned = df_cleaned.assign(Distance_Group=pd.cut(df_cleaned['Distance'], bins=bins, labels=labels, right=False))

# View Results
df_cleaned[['Distance', 'Distance_Group']].head()

In [ ]:
# Move 'Distance_Group' directly after 'Distance'
columns = df_cleaned.columns.to_list()

# Get the index of the 'Distance' column
distance_index = columns.index('Distance')

# Insert 'Distance Group' right after 'Distance'
columns.insert(distance_index + 1, columns.pop(columns.index('Distance_Group')))

# Update df_cleaned with new Column Order
df_cleaned = df_cleaned[columns]

# View Results
df_cleaned.head()

### Map each distance group category to an integer for sorting in PBI

In [ ]:
# Creating the mapping
distance_group_mapping = {
    "0-3 miles": 1,
    "3-5 miles": 2,
    "5-7 miles": 3,
    "7-10 miles": 4,
    "10-13 miles": 5,
    "13+ miles": 6
}

# Create the 'DistanceGroupId' column by mapping 'Distance_Group'
df_cleaned['DistanceGroupId'] = df_cleaned['Distance_Group'].map(distance_group_mapping)

# Move 'DistanceGroupId' directly after 'Distance_Group'
columns = df_cleaned.columns.to_list()

# Get the index of the 'Distance_Group' column
Distance_Group_index = columns.index('Distance_Group')

# Insert 'Distance Group' right after 'Distance_Group'
columns.insert(Distance_Group_index + 1, columns.pop(columns.index('DistanceGroupId')))

# Update df_cleaned with new Column Order
df_cleaned = df_cleaned[columns]

# View Results
df_cleaned.head()

In [ ]:
# Drop 'Favorite' Column as I have not been updating that field
df_cleaned = df_cleaned.drop('Favorite', axis = 1)
df_cleaned.head()

In [ ]:
# Convert 'Date' to datetime and set the time to 00:00:00
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date']).dt.normalize()

# Confirm Datatype Conversion
print(df_cleaned.dtypes)

# View Results
df_cleaned.head()

In [ ]:
# Drop Columns
cols_to_drop = ['Best_Lap_Time' # Usually only a few seconds as it takes me a second to stop my watch
                 ,'Number_of_Laps' # Directly Correlated with Miles 
                ,'Avg_GAP'] # I don't care about this metric

# View Results
df_cleaned = df_cleaned.drop(cols_to_drop,axis=1)

In [ ]:
# Create Week of Year Field
df_cleaned['Week_of_Year'] = df_cleaned['Date'].dt.isocalendar().week ## This starts the week on Monday and not Sunday

# # Adjust the week to start on Sunday by subtracting the weekday number from the date
# df_cleaned['Adjusted_Date'] = df_cleaned['Date'] - pd.to_timedelta(df_cleaned['Date'].dt.weekday + 1, unit='D')

# # Calculate week of the year based on the adjusted date (starting from Sunday)
# df_cleaned['Week_of_Year'] = df_cleaned['Adjusted_Date'].dt.isocalendar().week

# # Drop 'Adjusted_Date'
# df_cleaned = df_cleaned.drop('Adjusted_Date', axis=1)

# Create Month Field
df_cleaned['Month_Numeric'] = df_cleaned['Date'].dt.month

# Create Month Field with abbreviated month names
df_cleaned['Month'] = df_cleaned['Date'].dt.strftime('%b')

# Create Year Field
df_cleaned['Year'] = df_cleaned['Date'].dt.year

# View New Fields and Data Types
print(df_cleaned[['Date','Week_of_Year','Month','Year']].dtypes)
df_cleaned[['Date','Week_of_Year','Month','Year']].head()

In [ ]:
# Move 'Week_of_year','Month','Year' directly after 'Date'
columns = df_cleaned.columns.to_list()

# Get the index of the 'Weekly_Cumulative_Mins' column
distance_index = columns.index('Date')

# Insert 'Month' right after 'Date'
columns.insert(distance_index + 1, columns.pop(columns.index('Month_Numeric')))

# Insert 'Month' right after 'Date'
columns.insert(distance_index + 2, columns.pop(columns.index('Month')))

# Insert 'Year' right after 'Month'
columns.insert(distance_index + 3, columns.pop(columns.index('Year')))

# Insert 'Week_of_Year' right after 'Year'
columns.insert(distance_index + 4, columns.pop(columns.index('Week_of_Year')))

# Update df_cleaned with new Column Order
df_cleaned = df_cleaned[columns]

# View Results
df_cleaned.head()

In [ ]:
# View Time Column to ensure consistent formatting
df_cleaned['Time']

In [ ]:
# Function to drop milliseconds and keep only minutes and seconds
def drop_milliseconds(val):
    if isinstance(val, str) and '.' in val:
        # Keep only the minutes and seconds part, discard milliseconds
        minutes, _ = val.split('.')  # Drop the milliseconds
        return minutes  # Return only the minutes and seconds part
    return val  # Return original value if not in the expected format

# List of columns to clean
cols = ['Time', 'Avg_Pace', 'Best_Pace', 'Moving_Time', 'Elapsed_Time']

# Apply the function to the specified columns to drop milliseconds
for col in cols:
    df_cleaned[col] = df_cleaned[col].apply(drop_milliseconds)

# View dataframe
print(df_cleaned.dtypes)
df_cleaned[cols]

In [ ]:
# Function to convert 'mm:ss' to seconds
def convert_mmss_to_seconds(val):
    if isinstance(val, str) and ':' in val:
        minutes, seconds = val.split(':')
        return int(minutes) * 60 + int(seconds)
    return val  # Return the value if it's not in the expected format

# Check and convert 'Avg_Pace' column
df_cleaned['Avg_Pace'] = df_cleaned['Avg_Pace'].apply(convert_mmss_to_seconds)
df_cleaned['Avg_Pace'] = pd.to_timedelta(df_cleaned['Avg_Pace'], unit='s')

# Check and convert 'Best_Pace' column
df_cleaned['Best_Pace'] = df_cleaned['Best_Pace'].apply(convert_mmss_to_seconds)
df_cleaned['Best_Pace'] = pd.to_timedelta(df_cleaned['Best_Pace'], unit='s')
df_cleaned[['Avg_Pace','Best_Pace']].head()

### This needs to be a separate chunk because we are looking at hours as well

In [ ]:
# Function to standardize time format (convert hh:mm to 0:mm:ss)
def standardize_time_format(val):
    if isinstance(val, str):
        if ':' in val:
            parts = val.split(':')
            if len(parts) == 2:  # hh:mm format (2 parts)
                return f'0:{parts[0]}:{parts[1]}'  # Convert to 0:mm:ss format
            elif len(parts) == 3:  # hh:mm:ss format (3 parts)
                return val  # Already in the correct hh:mm:ss format
    return val  # Return original value if not a string or if it's already valid

# List of columns to apply the conversion to
cols = ['Time', 'Moving_Time', 'Elapsed_Time']

# Apply the function to the specified columns to standardize the time format
for col in cols:
    df_cleaned[col] = df_cleaned[col].apply(standardize_time_format)

# Convert the columns to timedelta
df_cleaned['Time'] = pd.to_timedelta(df_cleaned['Time'], errors='coerce')
df_cleaned['Moving_Time'] = pd.to_timedelta(df_cleaned['Moving_Time'], errors='coerce')
df_cleaned['Elapsed_Time'] = pd.to_timedelta(df_cleaned['Elapsed_Time'], errors='coerce')

# Create the 'Idle_Time' field by subtracting 'Moving_Time' from 'Elapsed_Time'
df_cleaned['Idle_Time'] = df_cleaned['Elapsed_Time'] - df_cleaned['Moving_Time']

# Display the updated dataframe
df_cleaned[cols]

In [ ]:
# Assess Data types after conversion
df_cleaned.dtypes

In [ ]:
# Assess Null Values
null_counts = df_cleaned.isna().sum()
print(null_counts)

In [ ]:
# Group by Year, Week and calculate cumulative sum
df_cleaned['Weekly_Cumulative_Mins'] = df_cleaned.groupby(['Year', 'Week_of_Year'])['Time'].cumsum()

# Calculate Weekly_Mins_Prior_to_Run by shifting the cumulative sum by one row
df_cleaned['Weekly_Mins_Prior_to_Run'] = df_cleaned.groupby(['Year', 'Week_of_Year'])['Weekly_Cumulative_Mins'].shift(1, fill_value=pd.Timedelta(0))

# Convert Timedelta to minutes
df_cleaned['Weekly_Mins_Prior_to_Run'] = df_cleaned['Weekly_Mins_Prior_to_Run'].dt.total_seconds() / 60

# Round the 'Weekly_Mins_Prior_to_Run' to 2 decimal places
df_cleaned['Weekly_Mins_Prior_to_Run'] = df_cleaned['Weekly_Mins_Prior_to_Run'].round(2)

# Display the updated DataFrame (optional)
print(df_cleaned[['Weekly_Mins_Prior_to_Run']].head())

In [ ]:
# Group by Year, Month and calculate cumulative sum
df_cleaned['Monthly_Cumulative_Mins'] = df_cleaned.groupby(['Year', 'Month'])['Time'].cumsum()

# Calculate Monthly_Mins_Prior_to_Run by shifting the cumulative sum by one row
df_cleaned['Monthly_Mins_Prior_to_Run'] = df_cleaned.groupby(['Year', 'Month'])['Monthly_Cumulative_Mins'].shift(1, fill_value=pd.Timedelta(0))

# Convert Timedelta to minutes
df_cleaned['Monthly_Mins_Prior_to_Run'] = df_cleaned['Monthly_Mins_Prior_to_Run'].dt.total_seconds() / 60

# Round the 'Monthly_Mins_Prior_to_Run' to 2 decimal places
df_cleaned['Monthly_Mins_Prior_to_Run'] = df_cleaned['Monthly_Mins_Prior_to_Run'].round(2)

df_cleaned.head()

In [ ]:
# Reorder Columns: 'Weekly_Cumulative_Mins','Weekly_Mins_Prior_to_Run','Monthly_Cumulative_Mins','Monthly_Mins_Prior_to_Run'
columns = df_cleaned.columns.to_list()

# Get the index of the 'Time' column
distance_index = columns.index('Time')

# Insert 'Weekly_Cumulative_Mins' right after 'Time'
columns.insert(distance_index + 1, columns.pop(columns.index('Weekly_Cumulative_Mins')))

# Insert 'Weekly_Mins_Prior_to_Run' right after 'Weekly_Cumulative_Mins'
columns.insert(distance_index + 2, columns.pop(columns.index('Weekly_Mins_Prior_to_Run')))

# Insert 'Monthly_Cumulative_Mins' right after 'Weekly_Mins_Prior_to_Run'
columns.insert(distance_index + 3, columns.pop(columns.index('Monthly_Cumulative_Mins')))

# Insert 'Monthly_Mins_Prior_to_Run' right after 'Monthly_Cumulative_Mins'
columns.insert(distance_index + 4, columns.pop(columns.index('Monthly_Mins_Prior_to_Run')))

# Update df_cleaned with new Column Order
df_cleaned = df_cleaned[columns]
df_cleaned.head()

In [ ]:
# Get the unique value count across all columns
unique_counts = df_cleaned.nunique()

# Display the unique value counts for each column
print(unique_counts)

In [ ]:
# Function to drop columns with only one unique value
def drop_single_value_columns(df):
    # Identify columns with only one unique value
    cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
    
    # Drop those columns
    df_cleaned = df.drop(cols_to_drop, axis=1)
    return df_cleaned

# Example usage
df_cleaned = drop_single_value_columns(df_cleaned)

# Display the dataframe after dropping the columns
df_cleaned.shape


### Conversion of Time-oriented fields to Duration in PBI was not working as expected
Drop '0 Days' for Time-oriented fields and format as strings for PBI

In [ ]:
# Function to convert timedelta to string and clean '0 days ' part
def clean_timedelta_to_string(val):
    if isinstance(val, pd.Timedelta):
        # Convert timedelta to string and remove '0 days ' part
        clean_value = str(val).split(' ')[-1]  # Keep only the 'hh:mm:ss' part
        return clean_value  # Return the string in hh:mm:ss format
    return val  # Return the original value if not a timedelta

# Create a new DataFrame df_pbi by copying df_cleaned
df_pbi = df_cleaned.copy()

# List of columns to clean
cols = ['Time', 'Avg_Pace', 'Weekly_Cumulative_Mins', 'Monthly_Cumulative_Mins', 'Best_Pace', 'Moving_Time', 'Elapsed_Time', 'Idle_Time']

# Apply the function to the specified columns in df_pbi to remove '0 days'
for col in cols:
    df_pbi[col] = df_pbi[col].apply(clean_timedelta_to_string)

# Check the data types after cleaning to ensure they are 'object' (string)
print(df_pbi.dtypes)

# View Results
df_pbi.head()


In [ ]:
df_pbi.dtypes

### Save Final Dataframe to a CSV for PBI

In [ ]:
# Save the DataFrame to a CSV file
df_pbi.to_csv('PBI Data/Running_Data_Cleaned_PBI.csv', index=False)

### Create new dataframe for PBI that is filtered to 18 Week training Plan Dates Only

In [ ]:
# Make Copy of Dataframe
training_plan = df_pbi.copy()

# Define the training start date (August 13, 2024)
training_start_date = pd.to_datetime('2024-08-13')

# Create the IsTrainingPlan column: 1 if the date is greater than or equal to training_start_date, otherwise 0
training_plan['IsTrainingPlan'] = (training_plan['Date'] >= training_start_date).astype(int)

# View Dataframe
training_plan[['Date','IsTrainingPlan']]

# Filter the DataFrame to include only rows where 'IsTrainingPlan' = 1
training_plan_filtered = training_plan[training_plan['IsTrainingPlan'] == 1]
training_plan_filtered

# Save the DataFrame to a CSV file
training_plan_filtered.to_csv('PBI Data/Training_Plan_PBI.csv', index=False)

## Sleep Data

In [ ]:
# Load in Sleep Data
sleep_1_main = pd.read_json('Raw Data/Sleep_Data/2023-10-15_2024-01-23_117832404_sleepData.json')
sleep_2_main = pd.read_json('Raw Data/Sleep_Data/2024-01-23_2024-05-02_117832404_sleepData.json')
sleep_3_main = pd.read_json('Raw Data/Sleep_Data/2024-05-02_2024-08-10_117832404_sleepData.json')

# Make Copies of Dataframes
sleep_1 = sleep_1_main.copy()
sleep_2 = sleep_2_main.copy()
sleep_3 = sleep_3_main.copy()

# View Imported File
sleep_1.head()

# Assess sleep_1 characteristcs
## print(sleep_1.shape)
## print(sleep_1.dtypes)

### Combine 3 Sleep Data Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_sleep = pd.concat([sleep_1, sleep_2, sleep_3], ignore_index=True)

# Convert 'calendarDate' to Datetime
combined_sleep['calendarDate'] = pd.to_datetime(combined_sleep['calendarDate'])

# View Dataframe
combined_sleep.head()

In [ ]:
# Convert the dictionary like values in 'sleepScores' column into separate columns
combined_sleep = combined_sleep.join(combined_sleep['sleepScores'].apply(pd.Series))

# Drop 'sleepScores' now that I have extracted information into other columns
combined_sleep = combined_sleep.drop('sleepScores',axis=1)

# View Dataframe
combined_sleep.head()

### Assess Null Values for Sleep Data

In [ ]:
# Count null values in each column
null_counts = combined_sleep.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# Convert the sleep start and end timestamps to datetime format
combined_sleep['sleepStartTimestampGMT'] = pd.to_datetime(combined_sleep['sleepStartTimestampGMT'])
combined_sleep['sleepEndTimestampGMT'] = pd.to_datetime(combined_sleep['sleepEndTimestampGMT'])

# Calculate the time difference between sleepStart and sleepEnd
combined_sleep['sleepDuration'] = combined_sleep['sleepEndTimestampGMT'] - combined_sleep['sleepStartTimestampGMT']

# Create a new column where sleepDuration is in float hours
combined_sleep['sleepDurationHours'] = (combined_sleep['sleepDuration'].dt.total_seconds() / 3600).round(1)  # Convert to hours as a float

# View Dataframe
combined_sleep.head()

In [ ]:
# Reorder Columns: 
columns = combined_sleep.columns.to_list()

# Get the index of the 'calendarDate' column
distance_index = columns.index('calendarDate')

# Remove 'sleepDurationHours' and 'sleepDuration' columns from the list
columns.remove('sleepDurationHours')
columns.remove('sleepDuration')

# Insert 'sleepDurationHours' right after 'calendarDate'
columns.insert(distance_index + 1, 'sleepDurationHours')

# Insert 'sleepDuration' right after 'sleepDurationHours'
columns.insert(distance_index + 2, 'sleepDuration')

# Reassign the new column order to the DataFrame
combined_sleep = combined_sleep[columns]

# Drop Columns
cols_to_drop = ['sleepStartTimestampGMT','sleepEndTimestampGMT','sleepWindowConfirmationType']
combined_sleep = combined_sleep.drop(cols_to_drop,axis=1)

# View Dataframe
combined_sleep.head()

In [ ]:
# Drop the last column by position using iloc
combined_sleep = combined_sleep.iloc[:, :-1]
combined_sleep.head()

In [ ]:
# Count null values in each column
null_counts = combined_sleep.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# View Rows in df where 'remSleepSeconds' is null
combined_sleep[combined_sleep['remSleepSeconds'].isna()]

In [ ]:
# Define a function to replace all null values with the column's average value for float64 datatype columns
def fill_null_with_mean(df, columns):
    for col in columns:
        mean_value = df[col].mean()
        df[col] = df[col].fillna(mean_value)  # Assign the filled column back to the DataFrame
    return df

# Get the list of float64 columns
float_columns = [col for col in combined_sleep.columns if combined_sleep[col].dtype == 'float64']

# Apply the function to replace null values with the mean
combined_sleep = fill_null_with_mean(combined_sleep, float_columns)

# View the DataFrame
combined_sleep

In [ ]:
# Count null values in each column
null_counts = combined_sleep.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# View rows where 'calendarDate' isna()
## combined_sleep[combined_sleep['calendarDate'].isna()]

# Manually assign the correct dates to the specific indices where 'calendarDate' is NaT
combined_sleep.loc[109, 'calendarDate'] = pd.Timestamp('2024-03-14')
combined_sleep.loc[257, 'calendarDate'] = pd.Timestamp('2024-08-09')
combined_sleep

In [ ]:
# Count null values in each column
null_counts = combined_sleep.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# Example: Make sure 'sleepDurationHours' does not have NaN values before applying
combined_sleep['sleepDuration'] = combined_sleep.apply(
    lambda row: pd.Timedelta(hours=row['sleepDurationHours']) if pd.isna(row['sleepDuration']) and pd.notna(row['sleepDurationHours']) else row['sleepDuration'],
    axis=1
)

combined_sleep

#### Re-format 'sleepDuration' so that the field can be converted to a duration dtype in PBI

In [ ]:
# Convert timedelta to string and remove '0 days '
combined_sleep['sleepDurationFormatted'] = combined_sleep['sleepDuration'].apply(lambda x: str(x).split(' ')[-1])

# Check the result
print(combined_sleep[['sleepDuration', 'sleepDurationFormatted']])

In [ ]:
# Reorder Columns
columns = combined_sleep.columns.to_list()

# Get the index of the 'sleepDuration' column
sleep_duration_index = columns.index('sleepDuration')

# Remove 'sleepDurationFormatted' column from the list
columns.remove('sleepDurationFormatted')

# Insert 'sleepDurationFormatted' right after 'sleepDuration'
columns.insert(sleep_duration_index + 1, 'sleepDurationFormatted')

# Reassign the new column order to the DataFrame
combined_sleep = combined_sleep[columns]

# View Dataframe
combined_sleep.head()

In [ ]:
# Round sleepDuration to seconds to remove microseconds and nanoseconds
combined_sleep['sleepDuration'] = combined_sleep['sleepDuration'].dt.round('s')  # Use 's' instead of 'S'

# If you want to format the duration as 'hh:mm:ss' without nanoseconds
combined_sleep['sleepDurationFormatted'] = combined_sleep['sleepDuration'].apply(lambda x: str(x).split(' ')[-1])

combined_sleep

In [ ]:
# Replace NaN values in 'insight' with "NONE"
combined_sleep['insight'] = combined_sleep['insight'].fillna("NONE")

# View value counts for 'feedback' categories
## combined_sleep['feedback'].value_counts()

# Replace NaN values in 'insight' with "NONE"
combined_sleep['feedback'] = combined_sleep['feedback'].fillna("NONE")

# View Dataframe
combined_sleep

### Convert columns LIKE '%Seconds%' to hours

In [ ]:
combined_sleep_cleaned = combined_sleep.copy()

# Convert seconds to hours (1 hour = 3600 seconds)
def seconds_to_hours(seconds):
    return round(seconds / 3600, 1)

# Identify columns that contain 'Seconds' in their name
columns_to_convert = [col for col in combined_sleep_cleaned.columns if 'Seconds' in col]

# Apply the conversion function to these columns
for col in columns_to_convert:
    combined_sleep_cleaned[col] = combined_sleep_cleaned[col].apply(seconds_to_hours)

combined_sleep_cleaned.head()

In [ ]:
# combined_sleep_cleaned[combined_sleep_cleaned['sleepDuration'].isna()] ## None

### Rename Columns

In [ ]:
def rename_seconds_to_hours(df):
    # Rename columns by replacing 'Seconds' with 'Hours'
    df = df.rename(columns={col: col.replace('Seconds', 'Hours') for col in df.columns if 'Seconds' in col})
    return df

# Apply the function to your DataFrame
combined_sleep_cleaned = rename_seconds_to_hours(combined_sleep_cleaned)

combined_sleep_cleaned.head()

# Confirm that there is 1 row per CalendarDate
## combined_sleep['calendarDate'].unique().value_counts()

In [ ]:
# Convert Floats to Int
# combined_sleep_cleaned = combined_sleep_cleaned.astype({col: 'int' for col in combined_sleep_cleaned.select_dtypes(include='float').columns})
# combined_sleep_cleaned.head()

# Drop Columns
cols_to_drop = ['retro','napList']
combined_sleep_cleaned = combined_sleep_cleaned.drop(cols_to_drop,axis=1)

# View Dataframe
combined_sleep_cleaned.head()

### Save Pre-Processed Sleep Data into a csv

In [ ]:
# Save the DataFrame to a CSV file
combined_sleep_cleaned.to_csv('PBI Data/Sleep_Cleaned_PBI.csv', index=False) #For PBI

## Load in Actute Training Load Data

In [ ]:
# Load in Acute Training Load Data
atl_1_main = pd.read_json('Raw Data/Acute_Training_Load/MetricsAcuteTrainingLoad_20231103_20240211_117832404.json')
atl_2_main = pd.read_json('Raw Data/Acute_Training_Load/MetricsAcuteTrainingLoad_20240211_20240521_117832404.json')
atl_3_main = pd.read_json('Raw Data/Acute_Training_Load/MetricsAcuteTrainingLoad_20240521_20240829_117832404.json')

# Make Copies of Dataframes
atl_1 = atl_1_main.copy()
atl_2 = atl_2_main.copy()
atl_3 = atl_3_main.copy()

# Assess atl_1 characteristcs
print(atl_1.shape)
print(atl_1.dtypes)

# View Imported File
atl_1
#atl_2
#atl_3

In [ ]:
# Filter out records with 'acwrStatus' == "NONE"\
atl_1_cleaned = atl_1[atl_1['acwrStatus'] != "NONE"]

print(atl_1_cleaned.shape)
atl_1_cleaned.head()

# atl_1_cleaned.shape ###(170,10) Dropped 34 of 204 records

### It appears the NONE records are from when I got the watch
### None of the records in atl_2 or atl_3 have NONE values for acwrStatus
### print(atl_2[atl_2['acwrStatus'] == "NONE"].shape)
### print(atl_3[atl_3['acwrStatus'] == "NONE"].shape)

### Assess Null Values in Acute Training Load Data

In [ ]:
# Count null values in each column
null_counts_1 = atl_1.isna().sum()
null_counts_2 = atl_2.isna().sum()
null_counts_3 = atl_3.isna().sum()

# Display the null counts
print(null_counts_1)
print(null_counts_2)
print(null_counts_3)

In [ ]:
# Assess Datatypes
atl_1_cleaned.dtypes

### Combine 3 Acute Training Load Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_atl = pd.concat([atl_1_cleaned, atl_2, atl_3], ignore_index=True)

# Change 'calendarDate' to Datetime
## combined_atl['calendarDate'] = pd.to_datetime(combined_atl['calendarDate']) ### Doesn't work as intended
combined_atl['calendarDate'] = pd.to_datetime(combined_atl['timestamp']).dt.date

### 'calendarDate' is in a really weird format so I am overriding it with the date from 'timestamp'
combined_atl.head()

In [ ]:
# Drop 'deviceId'
combined_atl = combined_atl.drop('deviceId', axis=1) ### Comment out after first execution

# View Results
combined_atl.head()

# Check for null values --> None
## combined_atl.isna().sum()

### Assess Null Values in Combined Dataframe

In [ ]:
combined_atl.isna().sum()

In [ ]:
# View Rows where dailyAcuteChronicWorkloadRatio is null
# combined_atl[combined_atl['dailyAcuteChronicWorkloadRatio'].isna()]

# Drop rows where dailyAcuteChronicWorkloadRatio is NaN
combined_atl_cleaned = combined_atl[combined_atl['dailyAcuteChronicWorkloadRatio'].notna()]
combined_atl_cleaned

### Filter to 1 record for each 'calendarDate'.
#### If there are multiple records for 1 Date, then keep the record with the maximum timestamp for that day

In [ ]:
# Group by 'calendarDate' and get the index of the row with the greatest 'timestamp' for each day
max_timestamp_idx = combined_atl_cleaned.groupby('calendarDate')['timestamp'].idxmax()

# Select the rows with the maximum 'timestamp' for each day
combined_atl_cleaned = combined_atl_cleaned.loc[max_timestamp_idx]

# Verify the result
combined_atl_cleaned

In [ ]:
# Drop 'acwrStatusFeedback' and 'timestamp' as these columns do not provide any value for ML model
combined_atl_cleaned = combined_atl_cleaned.drop(['userProfilePK','acwrStatusFeedback','timestamp'],axis=1)
combined_atl_cleaned.head()

### Save Pre-Processed Acute Training Load Data into a csv

In [ ]:
# Save the DataFrame to a CSV file
combined_atl_cleaned.to_csv('PBI Data/ATL_Cleaned_PBI.csv', index=False)

In [ ]:
combined_atl_cleaned

## Load in Max Met Data

In [ ]:
# Load in Max Met Data
maxmet_1_main = pd.read_json('Raw Data/Max_Met_Data/MetricsMaxMetData_20231103_20240211_117832404.json')
maxmet_2_main = pd.read_json('Raw Data/Max_Met_Data/MetricsMaxMetData_20240211_20240521_117832404.json')
maxmet_3_main = pd.read_json('Raw Data/Max_Met_Data/MetricsMaxMetData_20240521_20240829_117832404.json')

# Make Copies of Dataframes
maxmet_1 = maxmet_1_main.copy()
maxmet_2 = maxmet_2_main.copy()
maxmet_3 = maxmet_3_main.copy()

# View Imported File
maxmet_1
## maxmet_2
## maxmet_3

In [ ]:
# Assess maxmet_1 characteristcs
print(maxmet_1.shape)
print(maxmet_1.dtypes)

### Combine 3 Maxmet Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_maxmet = pd.concat([maxmet_1, maxmet_2, maxmet_3], ignore_index=True)

# Convert 'calendarDate' to Datetime
combined_maxmet['calendarDate'] = pd.to_datetime(combined_maxmet['calendarDate'])

combined_maxmet
## combined_maxmet

In [ ]:
print(combined_maxmet.shape)
print(combined_maxmet.dtypes)

### Assess Null Values for Maxmet Data

In [ ]:
# Count null values in each column
null_counts = combined_maxmet.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# Get the unique value count across all columns
unique_counts = combined_maxmet.nunique()

# Display the unique value counts for each column
print(unique_counts)

### Create Function to drop columns with only one unique value

In [ ]:
# Function to drop columns with only one unique value
def drop_single_value_columns(df):
    # Identify columns with only one unique value
    cols_to_drop = [col for col in df.columns if df[col].nunique() == 1]
    
    # Drop those columns
    df_cleaned = df.drop(cols_to_drop, axis=1)
    return df_cleaned

# Example usage
combined_maxmet = drop_single_value_columns(combined_maxmet)

# Display the dataframe after dropping the columns
combined_maxmet

### Filter to 1 record for each 'calendarDate'.
#### If there are multiple records for 1 Date, then keep the record with the maximum updateTimestamp for that day

In [ ]:
# Group by 'calendarDate' and get the index of the row with the greatest 'updateTimestamp' for each day
max_timestamp_idx = combined_maxmet.groupby('calendarDate')['updateTimestamp'].idxmax()

# Select the rows with the maximum 'timestamp' for each day
maxmet_cleaned = combined_maxmet.loc[max_timestamp_idx]

# Verify the result
print(combined_maxmet.shape)
print(maxmet_cleaned.shape)
maxmet_cleaned


### Add Records to Maxmet
#### If there is a date of 08-01 and the next records is 08-07, I want to duplicate the 08-01 record as 08-02,08-03, etc. to fill in the gaps until 08-07
#### This will allow me to have a row for every date when I join on 'calendarDate' later on to create my ML MASTER TBL

In [ ]:
# Create a complete date range from the first to the last date
date_range = pd.date_range(start=maxmet_cleaned['calendarDate'].min(), 
                           end=maxmet_cleaned['calendarDate'].max())

# Reindex the DataFrame with the new date range
maxmet_cleaned_2 = maxmet_cleaned.set_index('calendarDate').reindex(date_range)

# Forward fill the missing values to copy the previous day’s record
maxmet_cleaned_2 = maxmet_cleaned_2.ffill()

# Reset the index to bring 'calendarDate' back as a column
maxmet_cleaned_2 = maxmet_cleaned_2.reset_index().rename(columns={'index': 'calendarDate'})

# Verify the result
print(maxmet_cleaned.shape)
print(maxmet_cleaned_2.shape)
maxmet_cleaned_2

In [ ]:
# Drop 'updateTimestamp' and 'sport' as these columns do not provide much value
maxmet_cleaned_2 = maxmet_cleaned_2.drop(['updateTimestamp','sport'],axis=1)
maxmet_cleaned_2.head()

In [ ]:
# Fill missing 'vo2MaxValue' and 'maxMet' values with the value from the preceeding row
maxmet_cleaned_2.loc[:, 'vo2MaxValue'] = maxmet_cleaned_2['vo2MaxValue'].ffill()
maxmet_cleaned_2.loc[:, 'maxMet'] = maxmet_cleaned_2['maxMet'].ffill()

### This is not doing anything now, but if there are missing values later, this will fix the problem.
### May need to move this code prior to the generation of additional calendar Dates

### Save Pre-Processed Max Met Data into a csv

In [ ]:
maxmet_cleaned_2

In [ ]:
# Save the DataFrame to a CSV file
maxmet_cleaned_2.to_csv('PBI Data/MaxMet_Cleaned_PBI.csv', index=False)

## Load in Race Prediction Data

In [ ]:
# Load in Race Prediction Data
racepred_1_main = pd.read_json('Raw Data/Race_Predictions/RunRacePredictions_20231103_20240211_117832404.json')
racepred_2_main = pd.read_json('Raw Data/Race_Predictions/RunRacePredictions_20240211_20240521_117832404.json')
racepred_3_main = pd.read_json('Raw Data/Race_Predictions/RunRacePredictions_20240521_20240829_117832404.json')

# Make Copies of Dataframes
racepred_1 = racepred_1_main.copy()
racepred_2 = racepred_2_main.copy()
racepred_3 = racepred_3_main.copy()

# View Imported File
racepred_1.head()

# Preliminary Analysis

### There are multiple rows per Day. It may be best to take the average for the day. 
### Well maybe not because if the garmin algorithm is causing it to change intra-day, 
### then our algorithm should do the same thing. 
### Maybe start with a daily average and then progress from there

In [ ]:
# Assess racepred_1 characteristcs
print(racepred_1.shape)
print(racepred_1.dtypes)

### Combine 3 Race Prediction Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_racepred = pd.concat([racepred_1, racepred_2, racepred_3], ignore_index=True)

# Convert 'calendarDate' to Datetime
combined_racepred['calendarDate'] = pd.to_datetime(combined_racepred['calendarDate'])

# View Dataframe
combined_racepred.head()

### Clean up remaining data types for Race Predication Data

In [ ]:
# # Define a function to convert seconds to a timedelta
# def to_timedelta(seconds):
#     return pd.Timedelta(seconds=seconds)

# # List of columns to convert
# columns_to_convert = ['raceTime5K', 'raceTime10K', 'raceTimeHalf', 'raceTimeMarathon']

# # Apply the formatting function to each race time column
# for column in columns_to_convert:
#     combined_racepred[column] = combined_racepred[column].apply(to_timedelta)

In [ ]:
# # Define a function to convert seconds to a timedelta
# def to_timedelta(seconds):
#     return pd.Timedelta(seconds=seconds)

# # List of original columns to convert
# columns_to_convert = ['raceTime5K', 'raceTime10K', 'raceTimeHalf', 'raceTimeMarathon']

# # Create new columns with the converted values
# for column in columns_to_convert:
#     # Create new column names by appending '_timedelta' to the original column names
#     new_column_name = f"{column}_timedelta"
#     combined_racepred[new_column_name] = combined_racepred[column].apply(to_timedelta)

# # Display the dataframe with the new columns
# combined_racepred[['raceTime5K', 'raceTime5K_timedelta', 
#                    'raceTime10K', 'raceTime10K_timedelta', 
#                    'raceTimeHalf', 'raceTimeHalf_timedelta', 
#                    'raceTimeMarathon', 'raceTimeMarathon_timedelta']].head()

In [ ]:
combined_racepred.head()

In [ ]:
combined_racepred.dtypes

### Drop Invaluable columns from Race Prediction Data

In [ ]:
# Specify Columns to drop
columns_to_drop = ['deviceId'] ##, 'timestamp']

racepred_cleaned = combined_racepred.drop(columns_to_drop, axis=1)

racepred_cleaned.shape ## (917,7)

### Group by calendarDate and select the MIN race time for each category

In [ ]:
# Group by 'calendarDate' and find the minimum race times
min_race_times = racepred_cleaned.groupby('calendarDate').agg({
    'raceTime5K': 'min',
    'raceTime10K': 'min',
    'raceTimeHalf': 'min',
    'raceTimeMarathon': 'min'
}).reset_index()

min_race_times

In [ ]:

# Convert the race times to timedeltas (values are in seconds)
min_race_times['raceTime5K_timedelta'] = pd.to_timedelta(min_race_times['raceTime5K'], unit='s')
min_race_times['raceTime10K_timedelta'] = pd.to_timedelta(min_race_times['raceTime10K'], unit='s')
min_race_times['raceTimeHalf_timedelta'] = pd.to_timedelta(min_race_times['raceTimeHalf'], unit='s')
min_race_times['raceTimeMarathon_timedelta'] = pd.to_timedelta(min_race_times['raceTimeMarathon'], unit='s')

# View Results
min_race_times

## min_race_times['calendarDate'].unique().value_counts()
## min_race_times.shape (280,5)

### Conversion of Time-oriented fields to Duration in PBI was not working as expected
#### Drop '0 Days' for Time-oriented fields and format as strings for PBI

In [ ]:
# Function to convert timedelta to string and clean '0 days ' part
def clean_timedelta_to_string(val):
    if isinstance(val, pd.Timedelta):
        # Convert timedelta to string and remove '0 days ' part
        clean_value = str(val).split(' ')[-1]  # Keep only the 'hh:mm:ss' part
        return clean_value  # Return the string in hh:mm:ss format
    return val  # Return the original value if not a timedelta

# Create a new DataFrame df_pbi by copying df_cleaned
min_race_times_pbi = min_race_times.copy()

# List of columns to clean
cols = ['raceTime5K_timedelta','raceTime10K_timedelta','raceTimeHalf_timedelta','raceTimeMarathon_timedelta']

# Apply the function to the specified columns in df_pbi to remove '0 days'
for col in cols:
    min_race_times_pbi[col] = min_race_times_pbi[col].apply(clean_timedelta_to_string)

# Check the data types after cleaning to ensure they are 'object' (string)
print(min_race_times_pbi.dtypes)

# Display the updated dataframe
min_race_times_pbi.head()


In [ ]:
# Create the Year, Month, Quarter, and Day columns
min_race_times_pbi['Year'] = min_race_times_pbi['calendarDate'].dt.year
min_race_times_pbi['Month'] = min_race_times_pbi['calendarDate'].dt.month
min_race_times_pbi['MonthName'] = min_race_times_pbi['calendarDate'].dt.month_name()
min_race_times_pbi['Quarter'] = min_race_times_pbi['calendarDate'].dt.quarter
min_race_times_pbi['Day'] = min_race_times_pbi['calendarDate'].dt.day


# Check the resulting columns
print(min_race_times_pbi[['calendarDate', 'Year', 'Month', 'MonthName', 'Quarter', 'Day']].head())


In [ ]:
# Reorder Columns: 'Weekly_Cumulative_Mins','Weekly_Mins_Prior_to_Run','Monthly_Cumulative_Mins','Monthly_Mins_Prior_to_Run'
columns = min_race_times_pbi.columns.to_list()

# Get the index of the 'Time' column
distance_index = columns.index('calendarDate')

# Insert 'Year' right after 'calendarDate'
columns.insert(distance_index + 1, columns.pop(columns.index('Year')))

# Insert 'Month' right after 'Year'
columns.insert(distance_index + 2, columns.pop(columns.index('Month')))

# Insert 'MonthName' right after 'Month'
columns.insert(distance_index + 3, columns.pop(columns.index('MonthName')))

# Insert 'Quarter' right after 'Month'
columns.insert(distance_index + 4, columns.pop(columns.index('Quarter')))

# Insert 'Day' right after 'Quarter'
columns.insert(distance_index + 5, columns.pop(columns.index('Day')))

# Update df_cleaned with new Column Order
min_race_times_pbi = min_race_times_pbi[columns]

# Check the updated DataFrame structure
min_race_times_pbi.head()


### Save Pre-Processed Race Prediction Data into a csv

In [ ]:
# Save the DataFrame to a CSV file
min_race_times_pbi.to_csv('PBI Data/RacePredictions_Cleaned_PBI.csv', index=False)

## Loading in Training History Data
#### Do this after cleaning up Race Prediction Data for PBI

In [ ]:
# Load in Summarized Activity Data
training_hist_1_main = pd.read_json('Raw Data/Training_History/TrainingHistory_20231103_20240211_117832404.json')
training_hist_2_main = pd.read_json('Raw Data/Training_History/TrainingHistory_20240211_20240521_117832404.json')
training_hist_3_main = pd.read_json('Raw Data/Training_History/TrainingHistory_20240521_20240829_117832404.json')

# Make Copies of Dataframes
training_hist_1 = training_hist_1_main.copy()
training_hist_2 = training_hist_2_main.copy()
training_hist_3 = training_hist_3_main.copy()

# View Imported File
training_hist_3

# Preliminary Analysis

### There are multiple records per day as well. Need to factor these changes in.
### Maybe it is better to get a daily prediction, and then I can circle back to intra day updates.


### Combine 3 Training History Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_training_hist = pd.concat([training_hist_1, training_hist_2, training_hist_3], ignore_index=True)

# Convert 'calendarDate' to Datetime
combined_training_hist['calendarDate'] = pd.to_datetime(combined_training_hist['calendarDate'])

combined_training_hist

In [ ]:
# Assess combined_training_hist characteristcs
print(combined_training_hist.shape)
print(combined_training_hist.dtypes)

### Group By 'calendarDate' and Select the Last Training Status of Each Day

In [ ]:
# Group by 'calendarDate' and get the index of rows with the maximum timestamp
idx = combined_training_hist.groupby('calendarDate')['timestamp'].idxmax()

# Select the rows with the maximum timestamp for each day
combined_training_hist_cleaned = combined_training_hist.loc[idx]

# View Results
combined_training_hist_cleaned

# Confirm that there is only 1 row per 'calendarDate'
## combined_training_hist_cleaned['calendarDate'].unique().value_counts()

# Assess combined_training_hist_cleaned characteristics
## print(combined_training_hist_cleaned.shape) (280,9)
## print(combined_training_hist_cleaned.dtypes)

In [ ]:
# Count null values in each column
null_counts = combined_training_hist_cleaned.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# Drop records where 'trainingStatus' is 'NO_STATUS'
combined_training_hist_cleaned = combined_training_hist_cleaned[combined_training_hist_cleaned['trainingStatus'] != 'NO_STATUS']
combined_training_hist_cleaned

In [ ]:
# Drop Insignificant Columns
combined_training_hist_cleaned = combined_training_hist_cleaned.drop(['sport'
                                                                      ,'subSport'
                                                                      ,'deviceId'
                                                                      ,'timestamp'
                                                                      ,'trainingStatus2FeedbackPhrase'
                                                                      ,'userProfilePK']
                                                                      , axis=1)
combined_training_hist_cleaned.head()

In [ ]:
# Count null values in each column
null_counts = combined_training_hist_cleaned.isna().sum()

# Display the null counts
print(null_counts)

In [ ]:
# View Final Dataframe
combined_training_hist_cleaned

### Save Pre-Processed Training History Data into a csv

In [ ]:
# Save the DataFrame to a CSV file
combined_training_hist_cleaned.to_csv('PBI Data/TrainingHistory_Cleaned_PBI.csv', index=False)

## Load in UDS Data

In [ ]:
# Load in UDS Data
uds_1_main = pd.read_json('Raw Data/UDS_Data/UDSFile_2023-10-14_2024-01-22.json')
uds_2_main = pd.read_json('Raw Data/UDS_Data/UDSFile_2024-01-22_2024-05-01.json')
uds_3_main = pd.read_json('Raw Data/UDS_Data/UDSFile_2024-05-01_2024-08-09.json')

# Make Copies of Dataframes
uds_1 = uds_1_main.copy()
uds_2 = uds_2_main.copy()
uds_3 = uds_3_main.copy()

# View Imported File
uds_1.head()

# Assess uds_1 characteristcs
## print(uds_1.shape)
## print(uds_1.dtypes)

### Combine 3 UDS Dataframes into 1 Dataframe

In [ ]:
# Combine the three DataFrames into one
combined_uds = pd.concat([uds_1, uds_2, uds_3], ignore_index=True)

# Convert 'calendarDate' to Datetime
combined_uds['calendarDate'] = pd.to_datetime(combined_uds['calendarDate'])

combined_uds

# Assess combined_uds characteristcs
# print(combined_uds.shape)
# print(combined_uds.dtypes)

In [ ]:
# Count null values in each column
null_counts = combined_uds.isna().sum()

# Display the null counts
print(null_counts)

### Extract data from dictionary type columns

In [ ]:
# Convert the dictionary-like values in 'allDayStress' and specify suffixes to avoid column overlap
combined_uds = combined_uds.join(combined_uds['allDayStress'].apply(pd.Series), rsuffix='_stress')

# Drop Columns
combined_uds = combined_uds.drop(['allDayStress','calendarDate_stress','userProfilePK_stress'], axis=1)

# View Results
combined_uds.head()

In [ ]:
# Normalize each list of dictionaries in place without exploding the DataFrame
expanded_df = pd.json_normalize(combined_uds['aggregatorList'])

# Combine the new columns back into the original dataframe without altering row count
combined_uds = pd.concat([combined_uds, expanded_df], axis=1)

# Drop the original list column if necessary
combined_uds = combined_uds.drop(columns=['aggregatorList'])

# Drop Last 2 columns as the 3rd to last contains all necessary information
combined_uds = combined_uds.iloc[:, :-2]

# Rename '0' column 
combined_uds = combined_uds.rename(columns={combined_uds.columns[-1]: 'Total_stress_data'})
combined_uds.head()

# View Results
combined_uds.head()

In [ ]:
# Convert the dictionary like values in 'bodyBattery' column into separate columns
combined_uds = combined_uds.join(combined_uds['Total_stress_data'].apply(pd.Series), rsuffix='_stress')

# Drop 'Total_stress_data' Column
combined_uds = combined_uds.drop(['Total_stress_data'], axis=1)

combined_uds.head()
## combined_uds.shape (259,68)

### Not using this code at the moment

In [ ]:
# # Convert the dictionary like values in 'bodyBattery' column into separate columns
# combined_uds = combined_uds.join(combined_uds['bodyBattery'].apply(pd.Series), rsuffix='_battery')

# # Drop 'bodyBattery' Column
# combined_uds = combined_uds.drop(['bodyBattery'], axis=1)

# # Drop Other Duplicate Columns
# combined_uds = combined_uds.drop(['userProfilePK_battery','calendarDate_battery'], axis=1)
# combined_uds.head()

# combined_uds.head()

In [ ]:
# # Normalize each list of dictionaries in place without exploding the DataFrame
# expanded_df = pd.json_normalize(combined_uds['bodyBatteryStatList'])

# # Combine the new columns back into the original dataframe without altering row count
# combined_uds = pd.concat([combined_uds, expanded_df], axis=1)

# # Drop the original list column if necessary
# combined_uds = combined_uds.drop(columns=['bodyBatteryStatList'])

# combined_uds.head()
# ## combined_uds.shape (259,78)

In [ ]:
# Drop other insignificant columns
combined_uds = combined_uds.drop(['uuid'
                                  ,'userProfilePK'
                                  ,'wellnessStartTimeGmt'
                                  ,'wellnessEndTimeGmt'
                                  ,'wellnessStartTimeLocal'
                                  ,'wellnessEndTimeLocal'
                                  ,'includesWellnessData'
                                  ,'includesActivityData'
                                  ,'includesCalorieConsumedData'
                                  ,'includesSingleMeasurement'
                                  ,'includesContinuousMeasurement'
                                  ,'includesAllDayPulseOx'
                                  ,'includesSleepPulseOx'
                                  ,'source'
                                  ,'userFloorsAscendedGoal'
                                  ,'durationInMilliseconds'
                                  ,'wellnessKilocalories'
                                  ,'remainingKilocalories' # Same Values as 'wellnessKilocalories'
                                  ,'wellnessTotalKilocalories' # Same Values as 'wellnessKilocalories'
                                  ,'wellnessActiveKilocalories'
                                  ,'dailyStepGoal'
                                  ,'wellnessDistanceMeters'
                                  ,'userIntensityMinutesGoal'
                                  ,'minAvgHeartRate'
                                  ,'maxAvgHeartRate'
                                  ,'version'
                                  ,'restingCaloriesFromActivity'
                                  ,'restingHeartRateTimestamp'
                                 # ,'hydration'
                                  ,'dailyTotalFromEpochData'
                                  ,'type'
                                  ,'uncategorizedDuration'
                                  ,'totalDuration'
                                  ,'lowDuration'
                                  ,'bodyBattery'
                                  ,'floorsAscendedInMeters'
                                  ,'floorsDescendedInMeters'
                                  ,'averageMonitoringEnvironmentAltitude'
                                  ,'respiration'
                                  ,'hydration'
                                  ,'bodyBatteryFeedback'
                                 ], axis=1)
combined_uds.head()

In [ ]:
# Get the unique value count across all columns
unique_counts = combined_uds.nunique()

# Display the unique value counts for each column
print(unique_counts)

In [ ]:
# Replace NaN values with 0.0 in the 'isVigorousDay' column
combined_uds['isVigorousDay'] = combined_uds['isVigorousDay'].fillna(0.0)

# Verify the changes
combined_uds['isVigorousDay'].value_counts()

In [ ]:
combined_uds.isna().sum()

In [ ]:
# Fill missing values in each column with the column's mean
combined_uds = combined_uds.fillna(combined_uds.mean(numeric_only=True))
combined_uds

# Verify that the missing values have been filled
combined_uds.isna().sum() # This should show 0 for columns that had missing values

In [ ]:
# Covert all 'float' type columns to 'int'
combined_uds = combined_uds.astype({col: 'int' for col in combined_uds.select_dtypes(include='float').columns})
combined_uds

In [ ]:
# Create the Year, Month, Quarter, and Day columns
combined_uds['Year'] = combined_uds['calendarDate'].dt.year
combined_uds['Month'] = combined_uds['calendarDate'].dt.month
combined_uds['MonthName'] = combined_uds['calendarDate'].dt.month_name()
combined_uds['Quarter'] = combined_uds['calendarDate'].dt.quarter
combined_uds['Day'] = combined_uds['calendarDate'].dt.day


# Check the resulting columns
print(combined_uds[['calendarDate', 'Year', 'Month', 'MonthName', 'Quarter', 'Day']].head())


In [ ]:
# Reorder Columns: 'Weekly_Cumulative_Mins','Weekly_Mins_Prior_to_Run','Monthly_Cumulative_Mins','Monthly_Mins_Prior_to_Run'
columns = combined_uds.columns.to_list()

# Get the index of the 'Time' column
distance_index = columns.index('calendarDate')

# Insert 'Year' right after 'calendarDate'
columns.insert(distance_index + 1, columns.pop(columns.index('Year')))

# Insert 'Month' right after 'Year'
columns.insert(distance_index + 2, columns.pop(columns.index('Month')))

# Insert 'MonthName' right after 'Month'
columns.insert(distance_index + 3, columns.pop(columns.index('MonthName')))

# Insert 'Quarter' right after 'Month'
columns.insert(distance_index + 4, columns.pop(columns.index('Quarter')))

# Insert 'Day' right after 'Quarter'
columns.insert(distance_index + 5, columns.pop(columns.index('Day')))

# Update df_cleaned with new Column Order
combined_uds = combined_uds[columns]

# Check the updated DataFrame structure
combined_uds.head()


### Save Pre-Processed UDS Data into a csv

In [ ]:
# Save the DataFrame to a CSV file
combined_uds.to_csv('PBI Data/UDS_Cleaned_PBI.csv', index=False)